In [ ]:
from notebook_functions import Preprocess
from notebook_functions import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
%load_ext autoreload
%autoreload 2

train_import = pd.read_csv('../data/census_income_learn.csv', header=None)
test_import = pd.read_csv('../data/census_income_test.csv')

train = Preprocess(train_import, is_training=True)
train.preprocess()

test = Preprocess(test_import, is_training=False)
test.preprocess()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
test.X

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train.X, train.y)

In [5]:
# clf.predict(test.X)

In [ ]:
features = ['age', 'detailed_industry_recode', 'detailed_occupation_recode', 'education', 'marital_stat', 'sex',
        'tax_filer_stat', 
       'num_persons_worked_for_employer',  'citizenship',
       'own_business_or_self_employed',
       'weeks_worked_in_year',  'race_and_hispanic',
        'immigrant_score']


In [ ]:
X_train = pd.get_dummies(pd.concat([train.data[features], test.data[features]])).iloc[:train.data.shape[0]]
X_test = pd.get_dummies(pd.concat([train.data[features], test.data[features]])).iloc[train.data.shape[0]:]

In [ ]:
X_test.shape

In [ ]:
y_train = train.data['income']
y_test = test.data['income']

In [ ]:
clf = RandomForestClassifier(max_depth=20, random_state=0, min_samples_split=2, criterion='entropy')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)
GradientBoostingClassifier(random_state=0)
y_pred = gb.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from pycaret.classification import *

# Initialize setup (when using Notebook environment)
clf1 = setup(train.data, target = 'income')

# # Initialize setup (outside of Notebook environment)
# clf1 = setup(data, target = 'target-variable', html = False)

# # Initialize setup (When using remote execution such as Kaggle / GitHub actions / CI-CD pipelines)
# clf1 = setup(data, target = 'target-variable', html = False, silent = True)

In [ ]:
best = compare_models()


In [ ]:
clf1 = setup(train.data, target = 'income')

# train a decision tree model
dt = create_model('catboost')

# tune hyperparameters of decision tree
tuned_dt = tune_model(dt, optimize = 'Precision')


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import shap

explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)
